In [12]:
import pandas as pd 
import mlflow 
import mlflow.sklearn 
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import mean_absolute_error
from sklearn.datasets import fetch_california_housing

In [2]:
housing = fetch_california_housing()

In [5]:
print(housing.DESCR)

.. _california_housing_dataset:

California Housing dataset
--------------------------

**Data Set Characteristics:**

:Number of Instances: 20640

:Number of Attributes: 8 numeric, predictive attributes and the target

:Attribute Information:
    - MedInc        median income in block group
    - HouseAge      median house age in block group
    - AveRooms      average number of rooms per household
    - AveBedrms     average number of bedrooms per household
    - Population    block group population
    - AveOccup      average number of household members
    - Latitude      block group latitude
    - Longitude     block group longitude

:Missing Attribute Values: None

This dataset was obtained from the StatLib repository.
https://www.dcc.fc.up.pt/~ltorgo/Regression/cal_housing.html

The target variable is the median house value for California districts,
expressed in hundreds of thousands of dollars ($100,000).

This dataset was derived from the 1990 U.S. census, using one row per ce

In [6]:
## Preparing the dataset
data = pd.DataFrame(housing.data,columns=housing.feature_names)

In [8]:
data['Price'] = housing.target

In [9]:
data.head()

,MedInc,HouseAge,AveRooms,AveBedrms,Population,AveOccup,Latitude,Longitude,Price
0,8.3252,41.0,6.984127,1.023810,322.0,2.555556,37.88,-122.23,4.526
1,8.3014,21.0,6.238137,0.971880,2401.0,2.109842,37.86,-122.22,3.585
2,7.2574,52.0,8.288136,1.073446,496.0,2.802260,37.85,-122.24,3.521
3,5.6431,52.0,5.817352,1.073059,558.0,2.547945,37.85,-122.25,3.413
4,3.8462,52.0,6.281853,1.081081,565.0,2.181467,37.85,-122.25,3.422


In [10]:
# Train test split, Model Hyperparameters Tunning, MLFLOW Experiments

In [11]:
from urllib.parse import urlparse
## Independent and Dependent features
X = data.drop(columns=['Price'])
y = data['Price']

In [14]:
# Hyperparameters tunning using greadsearchcv 

def hyperparameters_tunning(X_train,y_train,param_grid):
    rf=RandomForestRegressor()
    grid_search = GridSearchCV(estimator=rf, param_grid=param_grid,
                               cv=3,n_jobs=-1, verbose=2,
                               scoring='neg_mean_squared_error')
    
    grid_search.fit(X_train,y_train)
    return grid_search

In [16]:
# Split data into training and testing
X_train,X_test,y_train,y_test = train_test_split(X,y, test_size=0.20)
from mlflow.models import infer_signature
signature = infer_signature(X_train,y_train)

## Defining hyper parameters for grid
param_grid = {
    'n_estimators': [100,200],
    'max_depth': [5,10,None],
    'min_samples_split':[2,5],
    'min_samples_leaf': [1,2]
}

## Start the mlflow

with mlflow.start_run():
    ## perform hyp tunning
    grid_search = hyperparameters_tunning(X_train,y_train,param_grid)
    
    ## Get the best models
    best_model =grid_search.best_estimator_

    ## Evaluate the best model
    
    y_pred = best_model.predict(X_test)
    mse = mean_absolute_error(y_test,y_train)
    
    
#Log tracking log and metrics
mlflow.log_param('best_n_estimators',grid_search.best_estimator_['n_estimators'])
mlflow.log_param('best_max_depth', grid_search.best_params_['max_depth'])
mlflow.log_param('best_min_samples_split', grid_search.best_params_['min_samples_split'])
mlflow.log_param('best_min_samples_leaf',grid_search.best_params_['min_samples_leaf'])
mlflow.log_metric('mse',mse)

# tracking Url
mlflow.set_tracking_uri(uri='http:/127.0.0.1:5000')
tracking_url_type_store = urlparse(mlflow.get_tracking_uri()).scheme

if tracking_url_type_store != 'file':
    mlflow.sklearn.log_model(best_model,'model',registered_model_name="Best Random Forest Model")
else:
    mlflow.sklearn.load_model(best_model,'model',signature=signature)
    
print(f'Best Hyperparameters: {grid_search.best_params}')
print(f'Mean Squared Error:{mse}')



Fitting 3 folds for each of 24 candidates, totalling 72 fits


ValueError: Found input variables with inconsistent numbers of samples: [4128, 16512]